In [11]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power
from timeit import default_timer as timer

In [12]:
D = 1000
U = np.zeros((D, 9))
V = np.zeros((D, 9))

In [13]:
for i in range(0, 9):
    U[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);
    V[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);

In [14]:
A = np.matmul(U,V.transpose())
eigenvalues, eigenvectors = LA.eig(A)
max_eigenvalue = max(np.absolute(eigenvalues))
A = np.dot(0.8, np.divide(A, max_eigenvalue))

In [15]:
mu = np.random.rand(D,1)/D
w = 1

In [16]:
import math
def comp_lambda(cur_t, cur_event, last_t, lambdat,w,mu,A):
    lambda_comp = mu + (lambdat - mu)*(math.exp(-w * (cur_t - last_t)))
    if (cur_event):
        lambda_comp = lambda_comp + np.expand_dims(A[cur_event, :].T,axis=1)
    return lambda_comp

In [17]:
import random 

def generate_samples(w, mu, A, num_sequences, max_events_per_sequence):
    start_time = timer()
    hp_samples = []

    for i in range(1, num_sequences+1):
        t = 0
        timestamp_and_event = []
        lambdat = mu
        lambdat_sum = np.sum(lambdat)
        
        while len(timestamp_and_event) < max_events_per_sequence:
            rand_u = random.uniform(0, 1)
            dt = np.random.exponential(1/lambdat_sum)            
            lambda_ts = comp_lambda(t+dt, [], t, lambdat,w,mu,A);
            lambdats_sum = np.sum(lambda_ts);
                        
            if (rand_u > (lambdats_sum/lambdat_sum)):
                t = t+dt
                lambdat = lambda_ts

            else:
                u = random.uniform(0, 1) * lambdats_sum
                lambda_sum = 0
                
                d = 0
                for d in range(1, len(mu)):
                    lambda_sum = lambda_sum + lambda_ts[d]
                    if(lambda_sum >= u):
                        break
            
                lambdat = comp_lambda(t+dt, d, t, lambdat, w, mu, A)
                t = t+dt
                timestamp_and_event.append([t,d])

            lambdat_sum = np.sum(lambdat)
        
        hp_samples.append(timestamp_and_event[0:])
        
        if (i%10 == 0):
            print("samples = " + str(i)+ "/" + str(num_sequences) + ", time = " 
                  + "{:.2f}".format(timer() - start_time) + " sec.")
    
    return hp_samples

In [18]:
num_sequences = 250
max_events_per_sequence = 100
hawkes_process_samples = generate_samples(w, mu, A, num_sequences, max_events_per_sequence)

samples = 10/250, time = 0.95 sec.
samples = 20/250, time = 1.82 sec.
samples = 30/250, time = 2.71 sec.
samples = 40/250, time = 3.63 sec.
samples = 50/250, time = 4.56 sec.
samples = 60/250, time = 5.46 sec.
samples = 70/250, time = 6.38 sec.
samples = 80/250, time = 7.36 sec.
samples = 90/250, time = 8.33 sec.
samples = 100/250, time = 9.32 sec.
samples = 110/250, time = 10.21 sec.
samples = 120/250, time = 11.14 sec.
samples = 130/250, time = 12.08 sec.
samples = 140/250, time = 13.12 sec.
samples = 150/250, time = 14.05 sec.
samples = 160/250, time = 14.99 sec.
samples = 170/250, time = 15.92 sec.
samples = 180/250, time = 17.06 sec.
samples = 190/250, time = 17.98 sec.
samples = 200/250, time = 18.90 sec.
samples = 210/250, time = 19.87 sec.
samples = 220/250, time = 20.79 sec.
samples = 230/250, time = 21.70 sec.
samples = 240/250, time = 22.66 sec.
samples = 250/250, time = 23.55 sec.


In [20]:
'''
timestamp = 1x100 double
event = 1x100 event            
'''
print (len(hawkes_process_samples))
print (len(hawkes_process_samples[0]))
print (hawkes_process_samples[0])

250
100
[[4.116737422086826, 631], [6.363598039648207, 222], [7.4935901518638515, 257], [10.651915241856132, 976], [12.314821479418132, 767], [13.242714664631265, 730], [13.409905825291153, 890], [13.451557972481586, 819], [13.870818163264648, 604], [14.176724221922798, 862], [14.581258646588216, 642], [14.861047736500574, 572], [15.118362249356515, 600], [15.464081498417222, 706], [15.482365108522258, 514], [15.836521714404958, 399], [15.938211270691907, 680], [16.009119005693144, 640], [16.17935399860373, 315], [16.231956877307187, 776], [16.26456926426052, 563], [16.65723134794148, 754], [16.730710005035107, 537], [17.471705112842645, 203], [17.874161414025565, 685], [18.38726360753607, 288], [18.90200081114345, 123], [18.960031482591027, 206], [19.33072941750521, 219], [19.500916925149912, 181], [19.933774961822518, 470], [19.96732524158788, 419], [20.07313759397205, 252], [20.188701916366977, 862], [20.341531917921362, 151], [20.39117803332626, 299], [20.4614471629782, 109], [20.9

In [ ]:
def kernel_g(dt, w):
    g = np.multiply(w, math.exp(np.multiply(-w, dt)))
    if (g > 1):
        g = 0
    return g

In [ ]:
def kernel_int_g(dt, w):
    G = np.subtract(1, math.exp(np.multiply(-w, dt)))
    if (G < 0):
        G = 0
    return G

In [ ]:
def real_err(A, A_m):
    err_1 = np.divide(abs(A - A_m), A)
    # Question: why we need err_2? (Incomplete)
    err_2 = abs(A - A_m) 
    err_1
    return err

In [ ]:
def convert_inf(A, inf_A):
    for i in range(0, len(inf_A)):
        for j in range(0, len(inf_A[0])):
            if(inf_A[i][j]):
                A[i][j] = 0
    return A

In [ ]:
def optimize_mu(hp_samples, D, w, rho, num_iter_1, num_iter_2, thold, real_A):
    A_m = np.random.rand(D,D);
    eigenvalues_m, eigenvectors_m = LA.eig(A_m)
    max_eigenvalue_m = max(np.absolute(eigenvalues_m))
    A_m = np.dot(0.8, np.divide(A_m, max_eigenvalue_m))
    
    # reshape may required in mu_m
    mu_m = np.divide(np.random.rand(D, 1), D)
    UL = np.zeros((D, D))
    ZL = np.zeros((D, D))
    US = np.zeros((D, D))
    ZS = np.zeros((D, D))
    
    for i in range(0, num_iter_1 + 1):
        rho = rho * 1.1
        for j in range(0, num_iter_2 + 1):
            print ("No. " + str(i + 1) + " outter while iteration | No. " 
                   + str(j + 1) +  " inner while iteration")
            A_m, mu_m, RelErr = update_mu(A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A)
            Iteration_Err[j + 1, i + 1] = RelErr
        s, v, d = np.linalg.svd(np.add(A_m, US))
        v = np.subtract(v, thold / rho)
        if(v < 0):
            v = 0
        ZL = s * v * d.T
        UL = UL + (A_m - ZL)
        tmp = np.subtract(abs(np.add(A_m, US)), thold / rho) # may have error
        if (temp < 0):
            temp = 0
        ZS = (np.multiply(np.sign(np.add(A_m, US)), tmp))
        US = np.add(US, np.subtract(A_M, ZS))
        
    return A_m, mu_m, Iteration_Err

In [ ]:
def update_mu (A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A):
    num_samples = length(hp_samples)
    mu_numerator = np.zeros(D, 1)
    
    
    C = np.zeros((len(A_m), len(A_m[0])))
    A_Step = np.add(np.zeros((len(A_m), len(A_m[0]))), r * rho)
    B = np.add(np.add(zeros((len(A_m), len(A_m[0]))), np.multiply(rho, np.subtract(UL, ZL))), 
               np.multiply(rho, np.subtract(US, ZS)))
    
    for s in range(0, num_samples):
        cur_hp_samples = hp_samples[s]
        timestamp = [i[0] for i in cur_hp_samples]
        event = [i[1] for i in cur_hp_samples]
        tc = timestamp[len(timestamp) - 1]
        nc = len(event)
        dt = np.subtract(tc, timestamp)
        
        for i in range(0, nc):
            ui = event[i]
            ti = timestamp[i]
            int_g = kernel_int_g(dt, w)
            
            # Todo: modify matrix B (Incomplete)
            
            pii = []
            pij = []
            ag_arr = []
            
            if (i > 0):
                tj = timestamp[0 : i]
                uj = event[0 : i]
                kn_g = kernel_g(ti - tj, w)
                ag_arr = np.multiply(A_m[uj, ui], kn_g.T)
                
            pii = np.divide(mu_m[ui], mu_m[ui] + sum(ag_arr))
            
            if(i > 0):
                pij = np.divide(ag_arr, mu_m[ui] + sum(ag_arr))
                if (len(pij) != 0 && sum([sum(k) for k in pij]) > 0):
                    for j in range(0, len(uj)):
                        uuj = uj[j]
                        C[uuj, ui] = C[uuj, ui] - pij[j, :] ## (Incomplete) Question: what we have at the end ??? value or vector ??
            
            mu_numerator[ui] = np.add(mu_numerator[ui], pii)
            
    mu = np.divide(mu_numerator, np.add(np.zeros((D, 1)), tc))
    sqrt_eq = math.sqrt(np.subtract(matrix_power(B, 2), np.multiply(4, np.multiply(A_step, C))))
    A  = np.divide(np.add(np.multiply(-1, B), sqrt_eq), np.multiply(2, A_Step))
    RelErr = real_err(real_A, A)
    
    print ("non-zero in mu = " + np.count_nonzero(mu))
    print ("non-zero in C = "  + np.count_nonzero(C))
    print ("non-zero in B = "  + np.count_nonzero(B) + ", non-zero in sqrt = " + np.count_nonzero(sqrt_eq))
    print ("real error = " + "{:.4f}".format(RelErr) + ", correlation = " + "{:.4f}".format()  # (Incomplete)
           + "#non-zero in A = " + np.count_nonzero(A))
            
    A = np.nan_to_num(A)
    A = convert_inf(A, np.isinf(A))
    
            
            
    return A, mu, RelErr